In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

# Take the sign names in our collections data

In [2]:
signs = sorted(os.listdir("./Keypoint_Data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [3]:
signs[0]

'hello'

In [4]:
sign_id['hello']

0

# Create the x and y data


In [5]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [6]:
x_train.shape

(950, 30, 1662)

In [7]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [8]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [9]:
model.fit(x_train, y_train, epochs=1000) # run this line when data collection is done (this will train the model)

Epoch 1/1000
30/30 [==============================] - 5s 55ms/step - loss: 2.3341 - categorical_accuracy: 0.1200
Epoch 2/1000
30/30 [==============================] - 2s 55ms/step - loss: 2.2677 - categorical_accuracy: 0.2558
Epoch 3/1000
30/30 [==============================] - 2s 55ms/step - loss: 1.9747 - categorical_accuracy: 0.2674
Epoch 4/1000
30/30 [==============================] - 2s 55ms/step - loss: 2.1290 - categorical_accuracy: 0.1747
Epoch 5/1000
30/30 [==============================] - 2s 55ms/step - loss: 1.8091 - categorical_accuracy: 0.2811
Epoch 6/1000
30/30 [==============================] - 2s 55ms/step - loss: 1.3994 - categorical_accuracy: 0.4053
Epoch 7/1000
30/30 [==============================] - 2s 55ms/step - loss: 1.1752 - categorical_accuracy: 0.5063
Epoch 8/1000
30/30 [==============================] - 2s 55ms/step - loss: 0.9279 - categorical_accuracy: 0.5958
Epoch 9/1000
30/30 [==============================] - 2s 55ms/step - loss: 0.7853 - categorical_

30/30 [==============================] - 2s 55ms/step - loss: 0.1140 - categorical_accuracy: 0.9705
Epoch 73/1000
30/30 [==============================] - 2s 57ms/step - loss: 0.0953 - categorical_accuracy: 0.9684
Epoch 74/1000
30/30 [==============================] - 2s 58ms/step - loss: 0.1020 - categorical_accuracy: 0.9653
Epoch 75/1000
30/30 [==============================] - 2s 56ms/step - loss: 0.1214 - categorical_accuracy: 0.9611
Epoch 76/1000
30/30 [==============================] - 2s 56ms/step - loss: 0.1173 - categorical_accuracy: 0.9547
Epoch 77/1000
30/30 [==============================] - 2s 57ms/step - loss: 0.0826 - categorical_accuracy: 0.9705
Epoch 78/1000
30/30 [==============================] - 2s 56ms/step - loss: 0.0763 - categorical_accuracy: 0.9768
Epoch 79/1000
30/30 [==============================] - 2s 57ms/step - loss: 0.1813 - categorical_accuracy: 0.9326
Epoch 80/1000
30/30 [==============================] - 2s 56ms/step - loss: 0.1232 - categorical_accur

KeyboardInterrupt: 

In [6]:
del model

NameError: name 'model' is not defined

In [15]:
model.summary() # summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            408320    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 30, 64)            49408     
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 32)                61472     
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 12)                7

In [10]:
res = model.predict(x_test)

2/2 [==============================] - 0s 12ms/step


In [12]:
for x in range(50):
    print(signs[np.argmax(y_test[x])] == signs[np.argmax(res[x])])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False


In [13]:
print(49/50 * 100)

98.0


In [14]:
signs[np.argmax(res[0])]

'wait'

In [15]:
signs[np.argmax(y_test[0])]

'wait'

# save weights


In [16]:
model.save("Model.h5") # save model

In [17]:
model = tf.keras.models.load_model("./Model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./Model.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmpqpwt_rbw\assets


INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmpqpwt_rbw\assets


746504

In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 